In [1]:
from __future__ import print_function
%env KERAS_BACKEND=tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense
from tictactoe import TicTacToeGame
import random
import numpy as np

env: KERAS_BACKEND=tensorflow


C:\Users\wit543\Anaconda3\envs\deeplearning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
game = TicTacToeGame()

def state_to_number( state):
    return np.array(list(map(int,(state.replace('X','1').replace('O','2').replace(' ','0'))))).reshape((1,9))

def get_move(idx):
#     print(game.state[:idx-1])
#     print( game.state[idx:])
    return game.state[:idx-1] + game.player + game.state[idx:]



# Agent 1
model1 = Sequential()
model1.add(Dense(32, input_shape=(9,)))
model1.add(Dense(9, activation='softmax'))
model1.compile(loss=keras.losses.categorical_crossentropy, # loss function use for calculate the amount of error
              optimizer=keras.optimizers.Adadelta(), # optimizer for determine how to adjusting the error
              metrics=['accuracy'])

# Agent 2
model2 = Sequential()
model2.add(Dense(32, input_shape=(9,)))
model2.add(Dense(9, activation='softmax'))
model2.compile(loss=keras.losses.categorical_crossentropy, # loss function use for calculate the amount of error
              optimizer=keras.optimizers.Adadelta(), # optimizer for determine how to adjusting the error
              metrics=['accuracy'])
envs = []
actions = []
epochs = 1000
for epoch in range(epochs):
    while game.playable():
        allowed_moves = [i+1 for i in range(9) if game.state[i] == ' ']
        if game.player == 'X':
            move = model1.predict(state_to_number(game.state))
        else:
            move = model2.predict(state_to_number(game.state))
        m = max(move[0])
        array = [i for i, j in enumerate(move[0]) if j == m]
        move = array[0] +1
        allowed_moves = [i+1 for i in range(9) if game.state[i] == ' ']
        if move  not in array:
            move = random.choice([i+1 for i in range(9) if game.state[i] == ' '])
        game.make_move(move)
        actions.append(move)
        envs.append(state_to_number(game.state))
    actions = keras.utils.to_categorical(np.array(actions)-1, 9)
    if game.winner == 'X':
        model1.fit(np.array(envs).reshape(len(envs),9),np.array(actions),batch_size=len(envs),verbose=0)
    else:
        model2.fit(np.array(envs).reshape(len(envs),9),np.array(actions),batch_size=len(envs),verbose=0)
    envs = []
    actions = []
    game = TicTacToeGame()

In [3]:
# run game
game = TicTacToeGame()
while game.playable():
    allowed_moves = [i+1 for i in range(9) if game.state[i] == ' ']
    print(game.player,"turn")
    game.print_board()
    if game.player == 'X':
        move = model1.predict(state_to_number(game.state))
        m = max(move[0])
        array = [i for i, j in enumerate(move[0]) if j == m]
        move = array[0] +1
        allowed_moves = [i+1 for i in range(9) if game.state[i] == ' ']
        if move  not in array:
            move = random.choice([i+1 for i in range(9) if game.state[i] == ' '])
        game.make_move(move)
    else:
        allowed_moves = [i+1 for i in range(9) if game.state[i] == ' ']
        idx = int(input('Choose move for {}, from {} : '.format(game.player, allowed_moves)))
        if any([i==idx for i in allowed_moves]):
            game.make_move(idx)

X turn
       |   |   
    -----------
       |   |   
    -----------
       |   |   
O turn
       |   |   
    -----------
     X |   |   
    -----------
       |   |   
X turn
       |   |   
    -----------
     X | O |   
    -----------
       |   |   
O turn
       |   | X 
    -----------
     X | O |   
    -----------
       |   |   
X turn
       |   | X 
    -----------
     X | O |   
    -----------
       |   | O 
O turn
       |   | X 
    -----------
     X | O |   
    -----------
       | X | O 
